In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
from collections import Counter

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [230]:
#loading data
headlines = pd.read_csv('FiQA_train_ABSA_financial_headlines.tsv',delimiter='\t',header=0)
posts = pd.read_csv('FiQA_train_ABSA_financial_posts.tsv',delimiter='\t',header=0)
posts = posts.drop(['Unnamed: 0', 'id'],axis=1)
headlines = headlines.drop(['Unnamed: 0', 'id'],axis=1)
posts_text = posts.iloc[:,:1]
posts_score = posts.iloc[:,1:]
headlines_text = headlines.iloc[:,:1]
headlines_score = headlines.iloc[:,1:]
text_frames = [posts_text,headlines_text]
score_frames = [posts_score,headlines_score]
text = pd.concat(text_frames)
score = pd.concat(score_frames)
# text = pd.read_csv('reviews.txt', header=None)
# score = pd.read_csv('labels.txt', header=None)

In [231]:
#counting word frequency to create vocabulary
counter_ = Counter()
for txt in text.values:
    for word in txt[0].split(" "):
        counter_[word]+=1

In [232]:
print(len(counter_))

11384


In [237]:
#keeping only 1000 most commmon words, frequency of 1000th word can be used to see if we need ot increase or decrease this number
vocabulary = sorted(counter_,key=counter_.get,reverse = True)[:1000]

In [270]:
vocabulary[:50]

['to',
 'the',
 'a',
 'for',
 'in',
 'and',
 'of',
 'on',
 '',
 'is',
 '$AAPL',
 'as',
 'up',
 'this',
 '-',
 'at',
 'with',
 'be',
 '$FB',
 '$TSLA',
 'it',
 '$SPY',
 'I',
 'from',
 'will',
 'by',
 'but',
 'buy',
 '&',
 'down',
 'that',
 'are',
 'long',
 'new',
 'after',
 'more',
 'short',
 'not',
 'Tesco',
 'share',
 '$AMZN',
 'out',
 'day',
 'over',
 'all',
 'Barclays',
 'see',
 'like',
 'you',
 'FTSE']

In [239]:
#word to index, will need this mapping to create word vector 
word_to_index = dict()
for index,word in enumerate(vocabulary):
    word_to_index[word] = index

In [240]:
def get_word_vector(data):
    vector = np.zeros(len(vocabulary), dtype=np.int_)
    for word in data.split(' '):
        index = word_to_index.get(word, None)
        if index != None:
            vector[index]+=1
    return vector

In [241]:
text_vectors = np.zeros((len(text),len(vocabulary)),dtype=np.int_)
for index,(_,data) in enumerate(text.iterrows()):
    text_vectors[index] = get_word_vector(data[0])

In [271]:
text_vectors[:50]

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0]])

In [245]:
#converting scores into positive and negative classes
Y = (score>=0).astype(np.int_)
Y['sentiment score'].value_counts()

1    2407
0    1382
Name: sentiment score, dtype: int64

In [249]:
shuffle = np.arange(len(score))
np.random.shuffle(shuffle)
test_train_split = 0.9
train_split, test_split = shuffle[:int(len(score)*test_train_split)], shuffle[int(len(score)*test_train_split):]
trainX, trainY = text_vectors[train_split,:], to_categorical(Y.values[train_split].ravel(), 2)
testX, testY = text_vectors[test_split,:], to_categorical(Y.values[test_split].ravel(), 2)

In [250]:
trainX = tflearn.data_utils.pad_sequences(trainX, maxlen=100, value=0.)
testX = tflearn.data_utils.pad_sequences(testX, maxlen=100, value=0.)
trainY

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [251]:
# Network building
# tf.reset_default_graph()
# net = tflearn.input_data([None, 100])
# net = tflearn.embedding(net, input_dim=len(vocabulary), output_dim=128)
# net = tflearn.lstm(net, 128, dropout=0.8)
# net = tflearn.fully_connected(net, 2, activation='softmax')
# net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
#                          loss='categorical_crossentropy')

tf.reset_default_graph()
net = tflearn.input_data([None, 100])                          # Input
net = tflearn.fully_connected(net, 200, activation='ReLU')      # Hidden
net = tflearn.fully_connected(net, 25, activation='ReLU')
net = tflearn.fully_connected(net, 2, activation='softmax')   # Output
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')


In [252]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY,n_epoch=100,validation_set=0.1, show_metric=True,batch_size=32)

Training Step: 9599  | total loss: 0.23581 | time: 0.381s
| SGD | epoch: 100 | loss: 0.23581 - acc: 0.8935 -- iter: 3040/3069
Training Step: 9600  | total loss: 0.23032 | time: 1.388s
| SGD | epoch: 100 | loss: 0.23032 - acc: 0.8947 | val_loss: 0.99878 - val_acc: 0.6686 -- iter: 3069/3069
--


In [269]:
# (np.array(model.predict(testX))[:,0]>=0.5).astype(np.int_)

predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)
model.predict(testX)

Test accuracy:  0.68073878628


array([[  9.35732901e-01,   6.42670691e-02],
       [  6.54032946e-01,   3.45967114e-01],
       [  5.10051753e-03,   9.94899452e-01],
       [  2.45214644e-04,   9.99754727e-01],
       [  2.29075085e-02,   9.77092505e-01],
       [  7.82655656e-01,   2.17344314e-01],
       [  6.14294350e-01,   3.85705620e-01],
       [  4.69169667e-04,   9.99530792e-01],
       [  2.42730647e-01,   7.57269442e-01],
       [  2.32494390e-03,   9.97675121e-01],
       [  7.15987338e-03,   9.92840171e-01],
       [  4.74704355e-01,   5.25295675e-01],
       [  1.92965910e-01,   8.07034135e-01],
       [  4.28548604e-02,   9.57145214e-01],
       [  1.10031627e-02,   9.88996863e-01],
       [  3.77339780e-01,   6.22660220e-01],
       [  2.47691702e-02,   9.75230813e-01],
       [  1.01659760e-01,   8.98340285e-01],
       [  1.01580910e-04,   9.99898434e-01],
       [  3.02013097e-04,   9.99698043e-01],
       [  8.78914773e-01,   1.21085189e-01],
       [  6.92424834e-01,   3.07575166e-01],
       [  

In [304]:
# score.values[test_split]

In [297]:
#trying svm
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [298]:
data = []
_class=[]
for ele in text.values.tolist():
    data.append(ele[0])
for ele in Y.values.tolist():
    _class.append(ele[0])

data_train,data_test = data[:int(len(data)*0.9)],data[int(len(data)*0.9):]
_class_train,_class_test = _class[:int(len(_class)*0.9)],_class[int(len(_class)*0.9):]

vectorizer = TfidfVectorizer(min_df=5,max_df = 0.8,sublinear_tf=True,use_idf=True)
train_vectors = vectorizer.fit_transform(data_train)
test_vectors = vectorizer.transform(data_test)

In [306]:
classifier_linear = svm.SVC(kernel='linear',probability=True)
classifier_linear.fit(train_vectors, _class_train)
prediction_linear = classifier_linear.predict(test_vectors)
prob = classifier_linear.predict_proba(test_vectors)
accuracy_score(_class_test,prediction_linear)
prob

array([[ 0.38806094,  0.61193906],
       [ 0.38806094,  0.61193906],
       [ 0.51735219,  0.48264781],
       [ 0.1332865 ,  0.8667135 ],
       [ 0.1332865 ,  0.8667135 ],
       [ 0.38173828,  0.61826172],
       [ 0.98022755,  0.01977245],
       [ 0.14910786,  0.85089214],
       [ 0.02315172,  0.97684828],
       [ 0.02315172,  0.97684828],
       [ 0.12910188,  0.87089812],
       [ 0.0200183 ,  0.9799817 ],
       [ 0.0200183 ,  0.9799817 ],
       [ 0.20369208,  0.79630792],
       [ 0.20369208,  0.79630792],
       [ 0.10126564,  0.89873436],
       [ 0.10126564,  0.89873436],
       [ 0.78861357,  0.21138643],
       [ 0.0324797 ,  0.9675203 ],
       [ 0.31737623,  0.68262377],
       [ 0.95554533,  0.04445467],
       [ 0.85242267,  0.14757733],
       [ 0.89818328,  0.10181672],
       [ 0.20656942,  0.79343058],
       [ 0.95851057,  0.04148943],
       [ 0.69088831,  0.30911169],
       [ 0.16160096,  0.83839904],
       [ 0.25299289,  0.74700711],
       [ 0.7898797 ,

In [300]:
# Perform classification with SVM, kernel=linear
classifier_liblinear = svm.LinearSVC()
classifier_liblinear.fit(train_vectors, _class_train)
prediction_liblinear = classifier_liblinear.predict(test_vectors)
accuracy_score(_class_test,prediction_liblinear)

0.65963060686015829

In [301]:
# # Perform classification with SVM, kernel=rbf
classifier_rbf = svm.SVC()
classifier_rbf.fit(train_vectors, _class_train)
prediction_rbf = classifier_rbf.predict(test_vectors)
accuracy_score(_class_test,prediction_rbf)

0.60949868073878632